In [1]:
import numpy as np 
import pandas as pd
import datetime
import psycopg2

In [2]:

host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com"
database="dalmia_digital"
user="postgres"
password="dalmia123"

def connection_db(host, database, user, password):
    conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password)
    sql_query =  "SELECT * FROM dalmia_sales.sales_data where billing_date > '2021-04-01' and sold_to_party_code between 7000000 and 7099999"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df


def create_date_columns(df_list):#pass the dataframe in the list format 
    new_df = pd.DataFrame()
    for i in df_list:
        i['date'] = pd.to_datetime(i['billing_date'])
        i['Month_Year'] = pd.to_datetime(i['date']).dt.strftime('%Y-%m')
        i['Year'] = pd.to_datetime(i['date']).dt.year
        i['Quarter'] = i['date'].dt.to_period('Q')
        new_df = pd.concat([new_df,i], axis=0)

    return new_df

def return_yoy_df(host, database, user, password):
    print(1)
    df = connection_db(host, database, user, password)
    print(2)
    #df['Billing Date'] = df['Billing Date'].apply(timeseries_conversion)
    print(3)
    df_sales = create_date_columns([df])
    print(4)
    df_sales = df_sales[['sold_to_party_code','Month_Year','bill_quantity']]
    print(5)
    df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()
    print(7)
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    print(8)
    return pivot_table

In [3]:
df = connection_db(host, database, user, password)

/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_8842/1743272461.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [4]:
df.head()

,month,billing_date,sales_region,sales_zone,sales_type,manufacturing_plant,manufacturing_plant_name,plant,plant_name,sold_to_party_code,...,direct_depot,incoterms_inco,mode_of_transport,t_zone,tzone_desc,brand,product,bill_quantity,gross_bill_price,zbill_invoice_bill
0,April-2022,2022-04-23,East,BHS,Trade,1323,DCBL FACTORY-BOKARO,C093,"DCBL, BIHARSHARIF",7005256,...,Depot,FPX,Rail,8031012303,BIHARSHARIF,DSP,DSP,8.0,70560,69760.0
1,April-2022,2022-04-23,East,ODW,Trade,1320,DCBL FACTORY-RAJGANGPUR,C095,"DCBL, VEDVYAS",7000407,...,Depot,FPX,Road,7700362107,BISRA,DSP,DSP,2.5,20000,19750.0
2,April-2022,2022-04-23,East,ODW,Trade,1320,DCBL FACTORY-RAJGANGPUR,C095,"DCBL, VEDVYAS",7000407,...,Depot,FPX,Road,7700362107,BISRA,DALMIA,PCC,2.5,18750,18500.0
3,April-2022,2022-04-23,East,ODW,Trade,1320,DCBL FACTORY-RAJGANGPUR,C095,"DCBL, VEDVYAS",7000407,...,Depot,FPX,Road,7700362107,BISRA,DALMIA,PCC,2.5,18750,18500.0
4,April-2022,2022-04-23,East,ODW,Trade,1320,DCBL FACTORY-RAJGANGPUR,C095,"DCBL, VEDVYAS",7000393,...,Depot,FTP,Road,7700332102,BIRAMITRAPUR,DALMIA,PCC,2.0,14600,14400.0


### Selecting Important columns

In [5]:
df.columns

Index(['month', 'billing_date', 'sales_region', 'sales_zone', 'sales_type',
       'manufacturing_plant', 'manufacturing_plant_name', 'plant',
       'plant_name', 'sold_to_party_code', 'sold_to_party_name',
       'sold_to_party_district_name', 'ship_to_party_code',
       'ship_to_party_name', 'ship_to_party_district', 'invoice_no',
       'direct_depot', 'incoterms_inco', 'mode_of_transport', 't_zone',
       'tzone_desc', 'brand', 'product', 'bill_quantity', 'gross_bill_price',
       'zbill_invoice_bill'],
      dtype='object')

In [6]:
imp_cols = ['month', 'billing_date','sales_region','sold_to_party_code','sold_to_party_district_name','ship_to_party_code',
            'ship_to_party_district','brand', 'product', 'bill_quantity']
df_sales = df[imp_cols].copy()

In [7]:
df_sales.head()

,month,billing_date,sales_region,sold_to_party_code,sold_to_party_district_name,ship_to_party_code,ship_to_party_district,brand,product,bill_quantity
0,April-2022,2022-04-23,East,7005256,NALANDA,7005256,NALANDA,DSP,DSP,8.0
1,April-2022,2022-04-23,East,7000407,SUNDARGARH,7000407,SUNDARGARH,DSP,DSP,2.5
2,April-2022,2022-04-23,East,7000407,SUNDARGARH,77064261,SUNDARGARH,DALMIA,PCC,2.5
3,April-2022,2022-04-23,East,7000407,SUNDARGARH,77064261,SUNDARGARH,DALMIA,PCC,2.5
4,April-2022,2022-04-23,East,7000393,SUNDARGARH,7000393,SUNDARGARH,DALMIA,PCC,2.0


In [8]:
df_sales.shape

(4099005, 10)

### For seasonal dealers 

In [9]:

def return_yoy_df(df):
   
    df_sales = create_date_columns([df])
    print(4)
    df_sales = df_sales[['sold_to_party_code','Month_Year','bill_quantity']]
    print(5)
    #df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()
    print(7)
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    print(8)
    return pivot_table

In [10]:
pivot_table = return_yoy_df(df_sales)

4
5
7
8


In [11]:
pivot_table.head()

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,20.0,20.0,20.0,122.5,75.0,90.0,155.0,46.0,86.0
7000010,57.0,37.0,NaN,24.0,30.0,NaN,41.0,24.0,25.0,24.0,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,54.5,45.0,74.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
7000019,12.0,50.0,133.0,104.0,165.0,123.5,NaN,7.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,24.0,22.0,5.0,21.0,24.0,NaN,29.0,NaN,10.0


In [12]:
pivot_table.shape

(15557, 26)

In [13]:
pivot_table = pivot_table.fillna(0)
def count_zeros_before_first_nonzero(row):
    count = 0
    for value in row:
        if value == 0:
            count += 1
        else:
            break 
    return count

In [14]:
pivot_table['Month_new_user'] = pivot_table.apply(count_zeros_before_first_nonzero , axis = 1)

In [15]:
pivot_table.head()

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,Month_new_user
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,20.0,20.0,122.5,75.0,90.0,155.0,46.0,86.0,0
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,45.0,74.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7000022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,22.0,5.0,21.0,24.0,0.0,29.0,0.0,10.0,13


In [16]:
pivot_table = pivot_table[pivot_table['Month_new_user']<=12]
pivot_table.shape

(11675, 27)

In [22]:
#pivot_table.drop('Month_new_user', axis = 1, inplace = True)

### Getting the status of each dealer 

In [17]:
def count_trailing_zeros_before_end_index(df, end_index):
    counts = []
    for row in df.itertuples(index=False):
        count = 0
        for i in range(end_index - 1, -1, -1):
            if row[i] == 0:
                count += 1
            else:
                break
        counts.append(count)
    return counts

result_matrix = pd.DataFrame(index = pivot_table.index)
for i in range(len(pivot_table.columns)+1):
    string = str(i)
    result_matrix[string] = count_trailing_zeros_before_end_index(pivot_table, i)

In [18]:
result_matrix.drop('0' , axis = 1 , inplace = True)

In [19]:
result_matrix

,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,26,27
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7000010,0,0,1,0,0,1,0,0,0,0,...,1,2,3,4,5,6,7,8,0,1
7000015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,3,4,5,6,7
7000019,0,0,0,0,0,0,1,0,1,2,...,10,11,12,13,14,15,16,17,18,19
7000025,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,1,2,3,4,5,6,7,8,9,10,...,0,0,0,0,0,0,0,0,0,0
7036281,1,2,3,4,5,6,7,8,9,10,...,4,5,6,7,8,9,10,11,12,0
7036287,1,2,3,4,5,6,7,8,9,10,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def inactive(value):
    if value == 0: 
        return 'Active'
    elif value == 1: 
        return 'Active_1N'
    elif value == 2: 
        return 'Active_2N'
    elif value == 3: 
        return 'Active_3N'
    elif value>3 and value <=12:
        return 'Inactive'
    else: 
        return 'Churn'
    
for i in result_matrix.columns:
    result_matrix[i] = result_matrix[i].apply(inactive)


In [21]:
result_matrix['Month_new_user'] = pivot_table['Month_new_user']
result_matrix

,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,25,26,27,Month_new_user
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,...,Active,Active,Active,Active,Active,Active,Active,Active,Active_1N,0
7000010,Active,Active,Active_1N,Active,Active,Active_1N,Active,Active,Active,Active,...,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,Inactive,Active,Active_1N,0
7000015,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,...,Active,Active,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,0
7000019,Active,Active,Active,Active,Active,Active,Active_1N,Active,Active_1N,Active_2N,...,Inactive,Inactive,Churn,Churn,Churn,Churn,Churn,Churn,Churn,0
7000025,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,...,Active,Active,Active,Active,Active,Active,Active,Active,Active_1N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Active,Active,Active,Active,Active,Active,Active,Active,Active,12
7036281,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Active,12
7036287,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Active,Active,Active,Active,Active,Active,Active,Active,Active,12


In [22]:
result_matrix_copy_1 = result_matrix.copy()
for i in range(result_matrix.shape[0]):
    value = result_matrix_copy_1.iloc[i,27]
    #print(result_matrix_copy_1.iloc[:, :value])
    
    result_matrix_copy_1.iloc[i, :value] = '-'

In [23]:
result_matrix_copy_1.to_csv('../data/result_matrix_updated.csv')

In [24]:
y = result_matrix_copy_1[['25','26']].copy()
result_matrix = result_matrix_copy_1[['1','2','3','4','5','6','7','8','9','10','11','12','13',
                                      '14','15','16','17','18','19','20','21','22','23','24']]

In [25]:
def count_active_1n(row):
    return row.str.count('Active_1N').sum()
pivot_table['Active_1N_Count'] = result_matrix.apply(lambda row: count_active_1n(row), axis=1).astype(int)
def count_active_2n(row):
    return row.str.count('Active_2N').sum()
pivot_table['Active_2N_Count'] = result_matrix.apply(lambda row: count_active_2n(row), axis=1).astype(int)
def count_active_3n(row):
    return row.str.count('Active_3N').sum()
pivot_table['Active_3N_Count'] = result_matrix.apply(lambda row: count_active_3n(row), axis=1).astype(int)

def count_inactive(row):
    return row.str.count('Inactive').sum()
pivot_table['Inactive_Count'] = result_matrix.apply(lambda row: count_inactive(row), axis=1).astype(int)

In [26]:
pivot_table

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2023-01,2023-02,2023-03,2023-04,2023-05,Month_new_user,Active_1N_Count,Active_2N_Count,Active_3N_Count,Inactive_Count
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,75.0,90.00,155.00,46.0,86.0,0,0,0,0,0
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.00,0.00,0.0,24.0,0,5,1,1,4
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,0.0,0.00,0.00,0.0,0.0,0,1,1,1,1
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0,2,1,1,9
7000025,98.0,108.5,87.0,105.0,107.5,102.0,107.0,117.0,60.0,85.0,...,179.0,137.00,173.00,158.0,96.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.5,54.00,70.00,17.0,7.0,12,0,0,0,0
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,12,1,1,1,7
7036287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,112.0,111.00,146.00,200.0,171.0,12,0,0,0,0


In [44]:
pivot_table_23 = pivot_table[['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03']].copy()

In [45]:
pivot_table_23['total'] = pivot_table_23.sum(axis = 1)

In [46]:
pivot_table_23

Month Year,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,total
Sold To Party Code,,,,,,,,,,,,,
7000004,91.0,114.0,35.0,22.0,20.0,20.0,20.0,20.0,122.5,75.0,90.00,155.00,784.5
7000010,25.0,17.0,0.0,33.8,12.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,87.8
7000015,27.0,75.5,100.0,42.0,54.5,45.0,74.0,27.0,0.0,0.0,0.00,0.00,445.0
7000019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
7000025,60.0,100.0,161.0,175.0,75.0,95.0,48.0,108.8,73.0,179.0,137.00,173.00,1384.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,24.0,144.0,86.0,74.0,67.0,65.0,30.0,61.0,6.0,22.5,54.00,70.00,703.5
7036281,10.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,16.0
7036287,25.0,90.0,174.0,104.0,89.0,104.0,111.0,122.0,137.0,112.0,111.00,146.00,1325.0


### Seasonal Dealer 

In [30]:
df_a = create_date_columns([df_sales])
df_seasonal = df_a[['Month_Year',
                        'bill_quantity','sold_to_party_district_name']].groupby(['sold_to_party_district_name' , 'Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()

In [31]:
df_seasonal.columns = ['sold_to_party_district_name', 'Month_Year' , 'Sum']
season_pivot = pd.pivot_table(df_seasonal, values='Sum',columns='Month_Year', index = 'sold_to_party_district_name')
season_pivot

Month_Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05
sold_to_party_district_name,,,,,,,,,,,,,,,,,,,,,
24 PRAGANA NORTH,14482.90,9103.00,8518.85,8349.95,8355.7,7395.80,8340.45,7803.65,12212.75,11586.50,...,9258.60,9742.60,7850.65,9578.85,11246.10,9421.90,9802.65,7860.25,4280.40,4256.85
24 PRAGANA SOUTH,20494.75,10664.85,10153.05,9379.25,9059.2,7297.80,9178.75,10440.35,17104.05,17403.25,...,10525.80,10100.45,9790.65,11885.60,12820.20,11608.40,13698.95,20425.15,9574.55,8849.50
ALIPURDUAR,3339.90,3573.35,3685.35,2332.05,2132.5,2427.00,2819.30,4401.90,7555.90,6114.00,...,4981.25,4726.95,2778.80,3732.80,3439.90,4899.85,4003.65,6106.00,4381.65,4341.90
ANGUL,6942.25,5449.30,6433.35,5603.65,6982.9,6497.95,7571.55,6692.10,8560.25,8405.05,...,4314.60,6171.90,5480.35,6183.55,7375.55,7342.95,7711.80,10152.80,4827.20,4425.90
ANUPPUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Tripura,5085.65,3139.90,2598.05,3269.05,3534.3,3728.55,3952.05,3894.20,5334.20,5768.40,...,5007.90,4466.50,3461.05,5162.95,5831.25,6276.90,6478.00,7684.85,6178.30,3982.15
Wokha,560.00,555.00,280.00,190.00,214.0,100.00,175.00,100.00,150.00,175.00,...,175.00,125.00,150.00,150.00,60.00,125.00,75.00,150.00,150.00,75.00
YADADRI BHUVANAGIRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.00,120.00,108.00,95.50,126.50,132.55,161.00,124.00,100.50,95.00


In [32]:
df_seasonal_dealerwise = df_a[['Month_Year','sold_to_party_code',
                        'bill_quantity','sold_to_party_district_name']].groupby(['sold_to_party_code',
                                                                                 'sold_to_party_district_name' , 'Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()

In [33]:
df_seasonal_dealerwise.columns =['sold_to_party_code','sold_to_party_district_name','Month_Year','Sum']
df_seasonal_dealerwise

,sold_to_party_code,sold_to_party_district_name,Month_Year,Sum
0,7000004,BHADRAK,2021-04,255.5
1,7000004,BHADRAK,2021-05,85.0
2,7000004,BHADRAK,2021-06,207.0
3,7000004,BHADRAK,2021-07,102.0
4,7000004,BHADRAK,2021-08,73.5
...,...,...,...,...
224402,7040778,CHATRA,2023-05,25.0
224403,7040779,MUZAFFARPUR,2023-05,10.0
224404,7040781,SAHEBGANJ,2023-05,7.5
224405,7040783,SAMBALPUR,2023-05,10.0


In [34]:
df_seasonal_22 = df_seasonal[df_seasonal['Month_Year']<'2022-04']
df_seasonal_23 = df_seasonal[df_seasonal['Month_Year']>'2022-03']

In [35]:
top_sales = df_seasonal_23.groupby('sold_to_party_district_name')['Sum'].nlargest(5)

# Extract the month_year values for each district with top sales
month_year_top_sales = df_seasonal.loc[top_sales.droplevel(0).index, ['sold_to_party_district_name', 'Month_Year' , 'Sum']]


In [36]:
month_year_top_sales

,sold_to_party_district_name,Month_Year,Sum
14,24 PRAGANA NORTH,2022-06,12677.50
20,24 PRAGANA NORTH,2022-12,11246.10
12,24 PRAGANA NORTH,2022-04,10217.70
22,24 PRAGANA NORTH,2023-02,9802.65
17,24 PRAGANA NORTH,2022-09,9742.60
...,...,...,...
9248,Yavatmal,2023-04,2382.25
9249,Yavatmal,2023-05,2281.25
9247,Yavatmal,2023-03,1987.00
9246,Yavatmal,2023-02,1550.00


In [38]:
df_seasonal_dealerwise_23 = df_seasonal_dealerwise[df_seasonal_dealerwise['Month_Year']>'2022-04']

In [39]:
df_seasonal_dealerwise_23

,sold_to_party_code,sold_to_party_district_name,Month_Year,Sum
13,7000004,BHADRAK,2022-05,114.0
14,7000004,BHADRAK,2022-06,35.0
15,7000004,BHADRAK,2022-07,22.0
16,7000004,BHADRAK,2022-08,20.0
17,7000004,BHADRAK,2022-09,20.0
...,...,...,...,...
224402,7040778,CHATRA,2023-05,25.0
224403,7040779,MUZAFFARPUR,2023-05,10.0
224404,7040781,SAHEBGANJ,2023-05,7.5
224405,7040783,SAMBALPUR,2023-05,10.0


In [40]:
merged_df = pd.merge(df_seasonal_dealerwise_23, month_year_top_sales, on=['sold_to_party_district_name', 'Month_Year'], how='inner')

In [41]:
season_pivot_23 = pd.pivot_table(merged_df[['sold_to_party_code','Month_Year','Sum_x']], values='Sum_x',columns='Month_Year', index = 'sold_to_party_code')

In [42]:
season_pivot_23 = season_pivot_23.fillna(0)
season_pivot_23['sum'] = season_pivot_23.sum(axis = 1)

In [47]:
a = pd.merge(season_pivot_23['sum'], pivot_table_23['total'] ,left_index=True , right_index=True , how = 'inner' )

In [48]:
a['percentage']= 100*(a['sum']/a['total'])

In [49]:
a

,sum,total,percentage
7000004,516.5,784.50,65.838113
7000010,17.0,87.80,19.362187
7000015,145.0,445.00,32.584270
7000025,639.0,1384.80,46.143847
7000027,1701.6,3659.05,46.503874
...,...,...,...
7036277,511.0,873.00,58.533792
7036279,373.0,703.50,53.020611
7036287,568.0,1325.00,42.867925
7036291,877.0,1551.00,56.544165


In [51]:
seasonal_result_23 = result_matrix[['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']].copy()
seasonal_result_23['Count_no_transaction'] = seasonal_result_23.apply(lambda row: count_active_1n(row), axis=1).astype(int)+seasonal_result_23.apply(lambda row: count_active_2n(row), axis=1).astype(int)+seasonal_result_23.apply(lambda row: count_active_3n(row), axis=1).astype(int)+seasonal_result_23.apply(lambda row: count_inactive(row), axis=1).astype(int)


In [52]:
seasonal_result_23

,13,14,15,16,17,18,19,20,21,22,23,24,Count_no_transaction
Sold To Party Code,,,,,,,,,,,,,
7000004,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,0
7000010,Active,Active,Active_1N,Active,Active,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,8
7000015,Active,Active,Active,Active,Active,Active,Active,Active,Active_1N,Active_2N,Active_3N,Inactive,4
7000019,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Churn,Churn,Churn,Churn,8
7000025,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,0
7036281,Active,Active,Active_1N,Active_2N,Active_3N,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,10
7036287,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,0


In [53]:
a = pd.merge(a , seasonal_result_23['Count_no_transaction'] , left_index=True , right_index=True , how = 'inner')

In [55]:
a

,sum,total,percentage,Count_no_transaction
7000004,516.5,784.50,65.838113,0
7000010,17.0,87.80,19.362187,8
7000015,145.0,445.00,32.584270,4
7000025,639.0,1384.80,46.143847,0
7000027,1701.6,3659.05,46.503874,0
...,...,...,...,...
7036277,511.0,873.00,58.533792,0
7036279,373.0,703.50,53.020611,0
7036287,568.0,1325.00,42.867925,0
7036291,877.0,1551.00,56.544165,0


In [62]:
a['seasonal_dealer'] = (a['Count_no_transaction'] >= 4) & (a['percentage'] > 60)
a['seasonal_dealer'] = a['seasonal_dealer'].astype(int)

In [63]:
a['seasonal_dealer'].value_counts()

0    8389
1     528
Name: seasonal_dealer, dtype: int64

In [64]:
a.to_csv('../data/seasonal_sealer.csv')

In [217]:
a[a['percentage']>=60]

,sum,total,percentage,Count_no_transaction
7000019,402.0,595.0,67.563025,5
7000315,236.0,332.0,71.084337,4
7000318,79.0,109.0,72.477064,6
7000389,116.0,170.0,68.235294,6
7000395,80.0,132.0,60.606061,5
...,...,...,...,...
7034982,11.0,11.0,100.000000,4
7035001,27.0,27.0,100.000000,4
7035011,58.0,58.0,100.000000,4
7035053,40.0,40.0,100.000000,4


In [218]:
a.to_csv('../data/seasonal.csv')

In [219]:
b = pivot_table_22[pivot_table_22.index ==7000004]
import matplotlib.pyplot as plt 


## Two methods
1. Calculate Z-score for a dealer for month
2. Forcasting methods

In [220]:
df

KeyboardInterrupt: 

In [ ]:
df.head()

In [68]:
import pandas as pd
import numpy as np
from datetime import datetime
import psycopg2
import streamlit as st
@st.cache_data

## Read all the require data from DB
def read_all():
    conn_string = "host='database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com' dbname='dalmia_digital' user='postgres' password='dalmia123'"
    conn_2 = psycopg2.connect(conn_string)
    df1= pd.read_sql_query("select sales_region, sales_zone,sold_to_party_district_name, billing_date, sold_to_party_code,sales_type, bill_quantity from dalmia_sales.sales_data where (billing_date > '2022-01-01' AND brand = 'DALMIA') AND (sold_to_party_code > 7*10^6 AND sold_to_party_code < 71*10^5) AND (bill_quantity >= 0 AND sales_type = 'Trade')",  conn_2)
    df2= pd.read_sql_query("select * from dalmia_sales.sales_pd_wsp", conn_2)
    df3= pd.read_sql_query("select * from dalmia_sales.south_sales_pd_wsp", conn_2)
    return df1,df2,df3

## Data Cleaning 
@st.cache_data
def datamanipulation(df1,df2,df3):
    
    ## Sales Data
    d1=df1.copy()
    d1.rename(columns={'sales_region': 'Sales_Region', 'sold_to_party_district_name' : 'District', 'billing_date': 'Date', 'bill_quantity': 'Volume','sold_to_party_code':'Dealers'}, inplace=True)
    d1.drop('sales_zone', axis=1, inplace=True)

    a1 = d1.groupby(['Sales_Region','District', 'Date'],as_index=False)['Dealers'].count()
    a2 = d1.groupby(['Sales_Region','District', 'Date'],as_index=False)['Volume'].sum()
    a3 = pd.merge(a1,a2, how='inner',left_on=['Sales_Region','District', 'Date'],right_on=['Sales_Region','District', 'Date'])
    # a3['Volume'] = a2.groupby(['Sales_Region','District'])['Volume'].fillna(method='ffill').fillna(method='bfill')
    a3.sort_values(by=['Sales_Region','District', 'Date'], inplace=True)
    a3['Sales_Region'] = a3['Sales_Region'].replace('North East', 'NE')
    a3['District'] = a3['District'].str.upper()
    a3['Volume'] = a3['Volume']*12
    
    ## WSP DATA: East
    d2=df2.copy()
    selected_columns = ['state', 'sh_location', 'district_code', 'district', 'date', 'dal_wsp', 'ut_wsp']
    d2 = d2[selected_columns].copy()
    d2.rename(columns={'state': 'State', 'sh_location' : 'SH_Location', 'district_code': 'District_Code', 'district': 'District', 'date':'Date', 'dal_wsp':'DAL WSP', 'ut_wsp':'UT WSP'}, inplace=True)
    d2['DAL WSP'] = d2.groupby(['State','SH_Location', 'District', 'Date'])['DAL WSP'].transform('sum')
    d2['UT WSP'] = d2.groupby(['State','SH_Location', 'District', 'Date'])['UT WSP'].transform('sum')
    def convert_to_integer(value):
        if isinstance(value, str) and '-' in value:
        # If the value is a string and '-' is present, split the value and check if it has a valid format
            parts = value.split('-')
            if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                start, end = int(parts[0]), int(parts[1])
                return int((start + end) / 2)
        # If the value is either an integer or a string without '-', convert it directly into an integer
        try:
            return int(value)
        except ValueError:
            return None
    d2['DAL WSP'] = d2['DAL WSP'].apply(convert_to_integer)
    d2['UT WSP'] = d2['UT WSP'].apply(convert_to_integer)
    # Deal with NaN
    d2['DAL WSP'] = d2.groupby(['State', 'SH_Location', 'District_Code', 'District'])['DAL WSP'].fillna(method='ffill').fillna(method='bfill')
    d2['UT WSP'] = d2.groupby(['State', 'SH_Location', 'District_Code', 'District'])['UT WSP'].fillna(method='ffill').fillna(method='bfill')
    d2.sort_values(['State', 'SH_Location', 'District_Code', 'District', 'Date'], inplace=True)
    
    ## WSP Data: South
    d3 = df3[df3['segment']=='Trade'].copy() 
    d3['billing_date']=pd.to_datetime(d3['billing_date'], infer_datetime_format=True, format='mixed' )
    d3['SH_Location']=df3['state']
    # State re-define
    d3['state'] = d3['state'].replace('Karnataka North','Karnataka')
    d3['state'] = d3['state'].replace('Karnataka South','Karnataka')
    d3['state'] = d3['state'].replace('TN1','Tamil Nadu')
    d3['state'] = d3['state'].replace('TN2','Tamil Nadu')
    d3['state'] = d3['state'].replace('TN3','Tamil Nadu')
    d3['state'] = d3['state'].replace('Maharashtra-A','Maharashtra')
    d3['state'] = d3['state'].replace('Maharashtra-B','Maharashtra')

    # SH Location re-define
    d3['SH_Location'] = d3['SH_Location'].replace('Karnataka North','KAN')
    d3['SH_Location'] = d3['SH_Location'].replace('Karnataka South','KAS')
    d3['SH_Location'] = d3['SH_Location'].replace('Telangana','TLG')
    d3['SH_Location'] = d3['SH_Location'].replace('Maharashtra-A','MHA')
    d3['SH_Location'] = d3['SH_Location'].replace('Maharashtra-B','MHB')
    d3['SH_Location'] = d3['SH_Location'].replace('Maharashtra','MH')
    d3['SH_Location'] = d3['SH_Location'].replace('Andhra Pradesh','ANDH PRD')
    d3['SH_Location'] = d3['SH_Location'].replace('Andaman','Others-S')

    selected_columns = ['state', 'SH_Location', 'district', 'billing_date', 'dalmia_wsp', 'ut_wsp']
    d3 = d3[selected_columns].copy()
    d3.rename(columns={'state': 'State', 'sh_location' : 'SH_Location', 'district': 'District', 'billing_date':'Date', 'dalmia_wsp':'DAL WSP', 'ut_wsp':'UT WSP'}, inplace=True)
    d3['DAL WSP'] = d3.groupby(['State','SH_Location', 'District', 'Date'])['DAL WSP'].transform('sum')
    d3['UT WSP'] = d3.groupby(['State','SH_Location', 'District', 'Date'])['UT WSP'].transform('sum')

    # Deal with NaN
    d3['DAL WSP'] = d3.groupby(['State', 'SH_Location', 'District'])['DAL WSP'].fillna(method='ffill').fillna(method='bfill')
    d3['UT WSP'] = d3.groupby(['State', 'SH_Location', 'District'])['UT WSP'].fillna(method='ffill').fillna(method='bfill')
    d3['District'] = d3['District'].str.upper()
    d3.sort_values(['State', 'SH_Location', 'District', 'Date'], inplace=True)
    
    ## Combine WSP Data and Sales Data 
    df_east = pd.merge(a3,d2, how='inner',left_on=['District', 'Date'],right_on=['District', 'Date'])
    df_east1= df_east[df_east['Sales_Region']=='East'].copy()
    df_south = pd.merge(a3,d3, how='inner',left_on=['District', 'Date'],right_on=['District', 'Date'])
    df_south1= df_south[df_south['Sales_Region']=='South'].copy()
    df = df_east1._append(df_south1, ignore_index=True)
    selected_cols= ['Sales_Region', 'State', 'SH_Location', 'District', 'Date', 'Dealers', 'DAL WSP', 'UT WSP', 'Volume']
    df= df[selected_cols].copy()
    df.sort_values(['Sales_Region', 'State', 'SH_Location', 'District', 'Date'], inplace=True)
        
    ## Price Index Caluclation Weekly
    # df_east['Year-Week'] = df_east['Date'].dt.strftime('%Y-%U')
    df['Year-Month'] = df['Date'].dt.strftime('%Y-%m')
    w1 = df.groupby(['Sales_Region','State', 'SH_Location', 'District', 'Year-Month'], as_index=False).apply(lambda group: pd.Series({
    'Monthly_UT WSP': ((group['UT WSP'] * group['Volume']).sum()) / group['Volume'].sum(),
    'Monthly_Volume': group['Volume'].sum(), 'Monthly_Dealers': group['Dealers'].sum()
    }))
    w2 = df.groupby(['Sales_Region','State', 'SH_Location', 'District', 'Year-Month'], as_index=False).apply(lambda group: pd.Series({
        'Monthly_DAL WSP': ((group['DAL WSP'] * group['Volume']).sum()) / group['Volume'].sum()
    }))
    w=pd.merge(w2,w1, how='inner',left_on=['Sales_Region', 'State', 'SH_Location', 'District','Year-Month'],right_on=['Sales_Region', 'State', 'SH_Location', 'District', 'Year-Month'])
    w.sort_values(by=['Sales_Region', 'State', 'SH_Location', 'District', 'Year-Month'], inplace=True)
    w['Price_Index'] = w['Monthly_DAL WSP']/w['Monthly_UT WSP']
    return w

2023-06-28 16:19:32.771 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-28 16:19:32.772 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [69]:
df1,df2,df3 = read_all()

2023-06-28 16:20:14.354 
  command:

    streamlit run /opt/homebrew/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-06-28 16:20:14.354 No runtime found, using MemoryCacheStorageManager
/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_8842/2953817311.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1= pd.read_sql_query("select sales_region, sales_zone,sold_to_party_district_name, billing_date, sold_to_party_code,sales_type, bill_quantity from dalmia_sales.sales_data where (billing_date > '2022-01-01' AND brand = 'DALMIA') AND (sold_to_party_code > 7*10^6 AND sold_to_party_code < 71*10^5) AND (bill_quantity >= 0 AND sales_type = 'Trade')",  conn_2)
/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_8842/2953817311.py:13: UserWarning: pandas only supports SQLAlchemy connectable (e

In [70]:
w = datamanipulation(df1,df2,df3)

2023-06-28 16:24:41.795 No runtime found, using MemoryCacheStorageManager
/var/folders/m3/m633mc4s5tsggxm0f81370l80000gn/T/ipykernel_8842/2953817311.py:63: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  d3['billing_date']=pd.to_datetime(d3['billing_date'], infer_datetime_format=True, format='mixed' )


In [72]:
w = w[['District', 'Year-Month','Price_Index']]

,District,Year-Month,Price_Index
0,ARARIA,2022-04,1.000000
1,ARARIA,2022-05,0.988655
2,ARARIA,2022-06,0.993778
3,ARARIA,2022-07,1.000000
4,ARARIA,2022-08,1.000000
...,...,...,...
2946,YADADRI BHUVANAGIRI,2022-11,0.940252
2947,YADADRI BHUVANAGIRI,2022-12,0.939481
2948,YADADRI BHUVANAGIRI,2023-01,0.938483
2949,YADADRI BHUVANAGIRI,2023-02,0.936909


In [74]:
pivot_table = pd.pivot_table(w, values='Price_Index',columns='Year-Month', index = 'District')
pivot_table

Year-Month,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04
District,,,,,,,,,,,,,,,,
24 PRAGANA NORTH,NaN,NaN,NaN,0.981824,1.004835,1.008451,1.000000,0.987966,0.995629,0.997320,0.991223,0.997336,0.995710,0.987153,0.986518,NaN
24 PRAGANA SOUTH,NaN,NaN,NaN,1.000000,0.998033,1.001697,0.994550,0.985509,0.995180,0.999323,0.996651,0.989986,0.992210,0.984450,0.985673,NaN
AHMEDNAGAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.944547,NaN,NaN,NaN,NaN,NaN,NaN
ALAPPUZHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999005,NaN,1.000000,1.000000,1.000000,1.000000
ALIPURDUAR,NaN,NaN,NaN,1.041509,1.067655,1.009963,0.949336,0.932432,0.975306,1.009288,1.032123,1.062377,1.058320,1.051603,0.990423,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WEST CHAMPARAN,NaN,NaN,NaN,1.055033,1.064481,1.033039,1.026760,1.026021,1.026555,1.027942,1.036522,1.038244,1.033933,1.051677,1.000000,NaN
WEST GODAVARI,1.0,1.0,1.0,1.000000,1.000000,0.999560,1.000000,1.000000,1.000000,1.000000,1.000000,0.999699,1.000000,1.000000,1.000000,1.000000
WEST SINGHBHUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.985075,NaN,NaN,1.012793,1.000000,NaN,1.009531,NaN
